# Imports

In [85]:
import numpy as np
import pandas as pd
import re
import nltk #pip install nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funciones

In [36]:
porter=nltk.PorterStemmer()

## Keyword

In [59]:
def clean_keyword(keyword):
    
    if keyword is not np.nan:
        keyword = re.sub("%20", " ", keyword)
        
        token_keyword = nltk.word_tokenize(keyword)
        new_keyword = []
        
        for word in token_keyword:
            if word not in stopwords.words('english'):
                new_keyword.append(porter.stem(word))

        keyword = " ".join(new_keyword)
        
    return keyword

## Location

In [141]:
def clean_location(location):
    if location is not np.nan:
        location = location.lower()
        location = re.sub("[^a-zñ, -]", "", location)
        location = re.sub("(^ +)|( +$)", "", location)
        location = re.sub("[,-]", " ", location)
        location = re.sub(r"\busa*\b", "united states of america", location)
        location = re.sub(r"\buk\b", "united kingdom", location)
        location = re.sub(r"\bny\b", "new york", location)
        location = re.sub(r"\bnyc\b", "new york city", location)
        location = re.sub(r"\bca\b", "california", location)
        location = re.sub(r"\bnc\b", "north carolina", location)
        location = re.sub(r"\btx\b", "texas", location)
        location = re.sub(r"\bga\b", "georgia", location)
        location = re.sub(r"\bncr\b", "national capital region of india", location)
        location = re.sub(r"\bfl\b", "florida", location)
        location = re.sub(r"\bsfo\b", "san francisco", location)
        location = re.sub(r"\bco\b", "colorado", location)
        location = re.sub(r"\boh\b", "ohio", location)
        location = re.sub(r"\bok\b", "oklahoma", location)
        
        location = re.sub(" +", " ", location)
    
    return location

## Text

In [4]:
def replace_text(text):
    text = text.replace('&amp', 'and')
    text = text.replace('amp;', 'and')
    text = text.replace('~', ' ')
    text = text.replace('Ûª', '\'')
    text = text.replace('ÛÒ', ' ')
    text = text.replace('ÛÓ', ' ')
    text = text.replace('&gt;', ' ')
    text = text.replace('&lt;', ' ')
    text = text.replace('ÛÏ', ' ')
    text = text.replace('Û', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('å«', '\'')
    text = text.replace('åÊ', ' ')
    text = text.replace('åÇ', ' ')
    text = text.replace('Ì©', 'e')
    text = text.replace('Ì¤', 'c')
    text = text.replace('Ì¼', 'u')
    text = text.replace('Ì_', 'o')
    text = text.replace('`', '\'' )
    text = text.replace('\x89', '')
    text = text.replace('\x9d', '')
    
    return text

In [33]:
one_letter="adhijklqruvwxyz"
double_letters="bcefgmnopst"

In [60]:
def clean_text(text):
    text = text.lower()
    
    text = re.sub("(http|https)\S+", "", text)
    text = re.sub(" can(\'*)t ", " cannot ", text)
    text = re.sub("n\'t ", " not ", text)
    text = re.sub("\'s ", " is ", text)
    text = re.sub("\'re ", " are ", text)
    text = re.sub("\'ve ", " have ", text)
    text = re.sub("\'d ", " would ", text)
    text = re.sub("\'m ", " am ", text)
    text = re.sub("\.|/|\+|-|÷|\*|:", " ", text)
    text = re.sub("( ' )|( # )", " ", text)
    text = re.sub("[^a-z 0-9]", "", text)
    
    
    for letter in one_letter:
        re_gex = letter + "(" + letter + "+)"
        text = re.sub(re_gex, letter, text)
        
    #Generalizamos para todas las palabras que contengan más de 2 letras iguales
    for letter in double_letters:
        double = letter + letter
        re_gex = double + "+"
        text = re.sub(re_gex, double, text)
    
    split_text = re.split("(\d+)", text)
    text = " ".join(split_text)
        
    text = re.sub("[0-9]\d*", "number", text)
    
    token_text = nltk.word_tokenize(text)    
    new_text = []
    for word in token_text:
        if word not in stopwords.words('english'):
            new_text.append(porter.stem(word))
    
    text = " ".join(new_text)
    text = re.sub(" +", " ", text) #Espacios múltiples
    
    return text

## Extras

In [7]:
english = 'abcdefghijklmnopqrstuvwxyz0123456789 '
len_english = len(english)

In [8]:
def search_text_samples(col_text):
    text = {}
    len_col_text = len(col_text)
    for i in range(0, len_col_text):
        len_text = len(col_text.iloc[i])
        for j in range(0, len_text):
            lower_case = col_text.iloc[i].lower()
            special_character = []
            if lower_case[j] not in english:
                if lower_case[j] in text:
                    special_character = text[lower_case[j]]
                special_character.append(i)
                text[lower_case[j]] = special_character
    return text

In [9]:
def show_text_samples(col_text, text):
    for key in text.keys():
        sample = text[key][0]
        print(col_text.iloc[sample])

# Train

In [142]:
train = pd.read_csv('train/train_original.csv')
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


## Keyword

In [70]:
train['keyword'].unique()[::5]

array([nan, 'ambulance', 'army', 'avalanche', 'blazing', 'blood',
       'body%20bags', 'buildings%20burning', 'bush%20fires',
       'chemical%20emergency', 'collided', 'crushed', 'dead', 'deluged',
       'derailed', 'destroyed', 'devastation', 'drowned', 'electrocuted',
       'epicentre', 'exploded', 'fatalities', 'first%20responders',
       'floods', 'harm', 'hijack', 'hurricane', 'inundation',
       'mass%20murder', 'military', 'obliterate', 'pandemonium',
       'quarantined', 'rescue', 'rubble', 'screams', 'sirens',
       'structural%20failure', 'survive', 'threat', 'trapped', 'twister',
       'war%20zone', 'wildfire', 'wreckage'], dtype=object)

In [71]:
train['keyword'] = train['keyword'].apply(clean_keyword)

In [72]:
train['keyword'].unique()

array([nan, 'ablaz', 'accid', 'aftershock', 'airplan accid', 'ambul',
       'annihil', 'apocalyps', 'armageddon', 'armi', 'arson', 'arsonist',
       'attack', 'avalanch', 'battl', 'bioterror', 'blaze', 'bleed',
       'blew', 'blight', 'blizzard', 'blood', 'bloodi', 'blown',
       'bodi bag', 'bomb', 'bridg collaps', 'build burn', 'build fire',
       'burn', 'burn build', 'bush fire', 'casualti', 'catastroph',
       'chemic emerg', 'cliff fall', 'collaps', 'collid', 'collis',
       'crash', 'crush', 'curfew', 'cyclon', 'damag', 'danger', 'dead',
       'death', 'debri', 'delug', 'demolish', 'demolit', 'derail',
       'desol', 'destroy', 'destruct', 'deton', 'devast', 'disast',
       'displac', 'drought', 'drown', 'dust storm', 'earthquak',
       'electrocut', 'emerg', 'emerg plan', 'emerg servic', 'engulf',
       'epicentr', 'evacu', 'explod', 'explos', 'eyewit', 'famin',
       'fatal', 'fear', 'fire', 'fire truck', 'first respond', 'flame',
       'flatten', 'flood', 'fores

In [73]:
len(train['keyword'].unique())

167

## Location

In [143]:
train['location'].unique()[::30]

array([nan, 'Santa Clara, CA', 'Oshawa, Canada', 'italy', 'Boston',
       'Elk Grove, CA, USA', 'Helsinki, Finland',
       'North-East Region, Singapore', 'West Bank, Gaza Strip',
       'The Hammock, FL, USA', 'Washington D.C.', 'Atlanta,Ga',
       'Swag Francisco', '?205?478?', 'Scotland', 'North Jersey',
       'Oklahoma', 'New Your', 'Menlo Park. SFO. The World.',
       'MY RTs ARE NOT ENDORSEMENTS', 'Pittsburgh PA',
       'taken by piper curda', '956', 'Whiterun, Skyrim',
       'Hoxton, London', 'Hartford  London Hong Kong', 'N?? Y???.',
       'Abuja, Nigeria', 'Pompano Beach, FL', 'Fort Smith, AR',
       'On the court ', '21.462446,-158.022017', 'San Antonio, TX',
       'Ankara - Malatya - ad Orontem', 'Hartford,  connecticut', 'USAoV',
       'UPTOWN ', 'the void, U.S.A', '518', 'Otsego, MI',
       'aggressive cannoli eater ', 'Anchorage, AK', 'Michigan, USA',
       'Johannesburg ', 'Patra-Greece.', "Sharkatraz/Bindle's Cleft, PA",
       'ITALY', 'UK  & Germany', 'Wh

In [144]:
train['location'] = train['location'].replace('M!$$!$$!PP!', 'Mississippi')
train['location'] = train['location'].apply(clean_location)

In [145]:
train['location'].unique()[::30]

array([nan, 'santa clara california', 'baker city oregon', 'toronto',
       'the own zone layer', 'austin texas', 'perthshire', 'atlanta',
       'ph', 'brasil', 'washington dc area', 'lima peru', 'indiana',
       'me mammys belly', 'biloxi mississippi', 'georgia', 'miami',
       'gia kardashianempire', 'puerto rico', 'new orleans louisiana',
       'belleville illinois', 'whiterun skyrim',
       'las vegas nv united states of america', 'wellington new zealand',
       'wales', 'draw a circle thats the earth', 'spokane wa',
       'protectingtitans side', 'th st ss', 'san diego texas',
       'illinois united states of america', 'columbia sc',
       'orlando florida', 'asuncin py tbingen ger', 'ct new york city',
       'fruit bowl', 'atlantic ia', 'coimbatore', 'chile',
       'georgia united states of america', 'the meadow',
       'nearest trash can', 'lynwood california', 'ct new york',
       'tchira venezuela', 'mumbai maharashtra',
       'united states of america northern 

In [146]:
len(train['location'].unique())

2897

## Text

In [79]:
text = search_text_samples(train['text'])
show_text_samples(train['text'], text)

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Forest fire near La Ronge Sask. Canada
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
13,000 people receive #wildfires evacuation orders in California 
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
What's up man?
What a goooooooaaaaaal!!!!!!
London is cool ;)
London is cool ;)
Cooool :)
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J
Barbados #Bridg

In [80]:
train['text'] = train['text'].apply(replace_text)

In [81]:
train['text'] = train['text'].apply(clean_text)

In [82]:
show_text_samples(train['text'], text)

deed reason earthquak may alah forgiv us al
forest fire near la rong sask canada
al resid ask shelter place notifi offic evacu shelter place order expect
number peopl receiv wildfir evacu order california
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir
man
gooal
london cool
london cool
cool
bbcmtd wholesal market ablaz
bbcmtd wholesal market ablaz
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend
outsid ablaz aliv dead insid
polic arsonist deliber set black church north 

## Resultado

In [83]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquak may alah forgiv us al,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,al resid ask shelter place notifi offic evacu ...,1
3,6,NaN,NaN,number peopl receiv wildfir evacu order califo...,1
4,7,NaN,NaN,got sent photo rubi alaska smoke wildfir pour ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant crane hold bridg colaps nearbi home,1
7609,10870,NaN,NaN,ariahrari thetawniest control wild fire califo...,1
7610,10871,NaN,NaN,number number number number utc number km volc...,1
7611,10872,NaN,NaN,polic investig e bike colid car littl portug e...,1


## Guardado del dataframe

In [84]:
train.to_csv('train/train_limpio.csv', index=False)

# Test

In [152]:
test = pd.read_csv('test/test_original.csv')

In [88]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


## Keyword

In [89]:
test['keyword'].unique()[::5]

array([nan, 'ambulance', 'army', 'avalanche', 'blazing', 'blood',
       'body%20bags', 'buildings%20burning', 'bush%20fires',
       'chemical%20emergency', 'collided', 'crushed', 'dead', 'deluged',
       'derailed', 'destroyed', 'devastation', 'drowned', 'electrocuted',
       'epicentre', 'exploded', 'fatalities', 'first%20responders',
       'floods', 'harm', 'hijack', 'hurricane', 'inundation',
       'mass%20murder', 'military', 'obliterate', 'pandemonium',
       'quarantined', 'rescue', 'rubble', 'screams', 'sirens',
       'structural%20failure', 'survive', 'threat', 'trapped', 'twister',
       'war%20zone', 'wildfire', 'wreckage'], dtype=object)

In [90]:
test['keyword'] = test['keyword'].apply(clean_keyword)

In [91]:
test['keyword'].unique()

array([nan, 'ablaz', 'accid', 'aftershock', 'airplan accid', 'ambul',
       'annihil', 'apocalyps', 'armageddon', 'armi', 'arson', 'arsonist',
       'attack', 'avalanch', 'battl', 'bioterror', 'blaze', 'bleed',
       'blew', 'blight', 'blizzard', 'blood', 'bloodi', 'blown',
       'bodi bag', 'bomb', 'bridg collaps', 'build burn', 'build fire',
       'burn', 'burn build', 'bush fire', 'casualti', 'catastroph',
       'chemic emerg', 'cliff fall', 'collaps', 'collid', 'collis',
       'crash', 'crush', 'curfew', 'cyclon', 'damag', 'danger', 'dead',
       'death', 'debri', 'delug', 'demolish', 'demolit', 'derail',
       'desol', 'destroy', 'destruct', 'deton', 'devast', 'disast',
       'displac', 'drought', 'drown', 'dust storm', 'earthquak',
       'electrocut', 'emerg', 'emerg plan', 'emerg servic', 'engulf',
       'epicentr', 'evacu', 'explod', 'explos', 'eyewit', 'famin',
       'fatal', 'fear', 'fire', 'fire truck', 'first respond', 'flame',
       'flatten', 'flood', 'fores

In [92]:
len(test['keyword'].unique())

167

## Location

In [153]:
len(test['location'].unique())

1603

In [148]:
test['location'].unique()[::50]

array([nan, 'USA ', 'Chicago, IL', 'East Texas', 'Oh.',
       '? the Foothills of SC ?', 'Somewhere on the Earth',
       'gateway regional hs', 'Leanbox?', 'The Open road!',
       'Toronto, Ontario, Canada', 'Camberville (Bostonish)',
       'Cardiff/London/NYC/Warwick', 'Please follow and RT! :)',
       'FEMA REGION 2', 'Isle of Patmos', 'West Palm Beach, FL',
       'Santa Cruz, CA', 'Desloge, Mo', 'Dark Night. ???? ??',
       'Limerick, Ireland.', 'Wickford', 'The Woodlands, TX',
       'trashcan somewhere in hell', '9.25.14?8.5.15?10.6.15 | gen?',
       ' Kaijo High School', 'Rice Lake, WI/Toronto, ON', 'NJ, USA',
       'St. Catharines', 'OKC', 'Okinawa', 'Sheffield.?',
       'Acey mountain islanddåÇTorontoåÈ'], dtype=object)

In [149]:
test['location'] = test['location'].apply(clean_location)

In [150]:
test['location'].unique()[::50]

array([nan, 'worldwide', 'reality based world',
       'denver colorado united states of america', 'twitchtvdgnesports',
       'dont stalk me thanks', 'cin city', 'mid north coast of nsw',
       'eugene or', 'piscataway nj', 'eastbourne east sussex',
       'st louis missouri', 'palacio madrid', 'jupiter',
       'paonia colorado', 'alexander iowa', 'lampe mo', 'stgo chile',
       'california central valley', 'beijing china', 'dili east timor',
       'theythem', 'colorado the mile high city', 'benua ltd',
       'london herts', 'virginia beach va united states of america',
       'portland oregon united states of america', 'louavul ky',
       'where eer the mood takes me'], dtype=object)

In [151]:
len(test['location'].unique())

1437

## Text

In [97]:
text = search_text_samples(test['text'])
show_text_samples(test['text'], text)

Heard about #earthquake is different cities, stay safe everyone.
Heard about #earthquake is different cities, stay safe everyone.
Heard about #earthquake is different cities, stay safe everyone.
We're shaking...It's an earthquake
They'd probably still show more life than Arsenal did yesterday, eh? EH?
Hey! How are you?
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
@sunkxssedharry will you wear shorts for race ablaze ?
#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI
#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI
#PreviouslyOnDoyinTv: Toke MakinwaÛªs ma

In [98]:
test['text'] = test['text'].apply(replace_text)

In [99]:
test['text'] = test['text'].apply(clean_text)

In [100]:
show_text_samples(test['text'], text)

heard earthquak differ citi stay safe everyon
heard earthquak differ citi stay safe everyon
heard earthquak differ citi stay safe everyon
shake earthquak
would probabl stil show life arsen yesterday eh eh
hey
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market
sunkxssedhari wil wear short race ablaz
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz
psa split person techi folow ablazeco burner folow ablaz
psa split person techi folow ablazeco burner folow ablaz
bewar world ablaz siera leon guap
bewar world ablaz siera leon guap
number car record zeroedg dual len car camera vehicl traffic drive histori accid camcord larg
number car recor

## Guardado del dataframe

In [101]:
test.to_csv('test/test_limpio.csv', index=False)